### Data Extraction of COVID-19 spread

In [1]:
from IPython.core.display import display, HTML,display_html
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np

In [3]:
confirmed_df= pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

In [4]:
confirmed_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,5639,6053,6402,6664,7072,7653,8145,8676,9216,9998
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,898,916,933,946,948,949,964,969,981,989
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,6442,6629,6821,7019,7201,7377,7542,7728,7918,8113
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,761,761,761,761,761,761,762,762,762,762
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,48,48,48,48,50,52,52,58,60,61


In [5]:
dates = confirmed_df.columns[4:]

confirmed_df_long = confirmed_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Confirmed'
)
deaths_df_long = deaths_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Deaths'
)
recovered_df_long = recovered_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Recovered'
)

In [6]:
confirmed_df_long

,Province/State,Country/Region,Lat,Long,Date,Confirmed
0,NaN,Afghanistan,33.000000,65.000000,1/22/20,0
1,NaN,Albania,41.153300,20.168300,1/22/20,0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0
...,...,...,...,...,...,...
32713,NaN,Sao Tome and Principe,0.186360,6.613081,5/23/20,251
32714,NaN,Yemen,15.552727,48.516388,5/23/20,212
32715,NaN,Comoros,-11.645500,43.333300,5/23/20,78
32716,NaN,Tajikistan,38.861034,71.276093,5/23/20,2738


In [7]:
## Remove Canada from df as recovered data is counted by Country-wise rather than Province/State-wise
recovered_df_long = recovered_df_long[recovered_df_long['Country/Region']!='Canada']

In [8]:
# Merging confirmed_df_long and deaths_df_long
full_table = confirmed_df_long.merge(
  right=deaths_df_long, 
  how='left',
  on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']
)
# Merging full_table and recovered_df_long
full_table = full_table.merge(
  right=recovered_df_long, 
  how='left',
  on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']
)

In [9]:
full_table['Date'] = pd.to_datetime(full_table['Date'])

In [10]:
## Check for missing values
full_table.isna().sum()

Province/State    22755
Country/Region        0
Lat                   0
Long                  0
Date                  0
Confirmed             0
Deaths                0
Recovered          3444
dtype: int64

In [11]:
full_table['Recovered'] = full_table['Recovered'].fillna(0)

In [12]:
## Identify data of ship rows
ship_rows = full_table['Province/State'].str.contains('Grand Princess') | full_table['Province/State'].str.contains('Diamond Princess') | full_table['Country/Region'].str.contains('Diamond Princess') | full_table['Country/Region'].str.contains('MS Zaandam')
full_ship = full_table[ship_rows]

In [13]:
## Remove rows containing COVID 19 spread on ships
full_table = full_table[~(ship_rows)]

In [14]:
# Active Case = confirmed - deaths - recovered
full_table['Active'] = full_table['Confirmed'] - full_table['Deaths'] - full_table['Recovered']
full_table

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
0,NaN,Afghanistan,33.000000,65.000000,2020-01-22,0,0,0.0,0.0
1,NaN,Albania,41.153300,20.168300,2020-01-22,0,0,0.0,0.0
2,NaN,Algeria,28.033900,1.659600,2020-01-22,0,0,0.0,0.0
3,NaN,Andorra,42.506300,1.521800,2020-01-22,0,0,0.0,0.0
4,NaN,Angola,-11.202700,17.873900,2020-01-22,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
32713,NaN,Sao Tome and Principe,0.186360,6.613081,2020-05-23,251,8,4.0,239.0
32714,NaN,Yemen,15.552727,48.516388,2020-05-23,212,39,0.0,173.0
32715,NaN,Comoros,-11.645500,43.333300,2020-05-23,78,1,18.0,59.0
32716,NaN,Tajikistan,38.861034,71.276093,2020-05-23,2738,44,0.0,2694.0


In [15]:
# aggregate data into Country/Region wise and group them by Date and Country/Region.
full_grouped = full_table.groupby(['Date', 'Country/Region'])[['Confirmed', 'Deaths', 'Recovered', 'Active']].sum().reset_index()
full_grouped

,Date,Country/Region,Confirmed,Deaths,Recovered,Active
0,2020-01-22,Afghanistan,0,0,0.0,0.0
1,2020-01-22,Albania,0,0,0.0,0.0
2,2020-01-22,Algeria,0,0,0.0,0.0
3,2020-01-22,Andorra,0,0,0.0,0.0
4,2020-01-22,Angola,0,0,0.0,0.0
...,...,...,...,...,...,...
22873,2020-05-23,West Bank and Gaza,423,3,348.0,72.0
22874,2020-05-23,Western Sahara,6,0,6.0,0.0
22875,2020-05-23,Yemen,212,39,0.0,173.0
22876,2020-05-23,Zambia,920,7,336.0,577.0


In [16]:
# new cases 
temp = full_grouped.groupby(['Country/Region', 'Date' ])[['Confirmed', 'Deaths', 'Recovered']]
temp = temp.sum().diff().reset_index()
mask = temp['Country/Region'] != temp['Country/Region'].shift(1)
temp.loc[mask, 'Confirmed'] = np.nan
temp.loc[mask, 'Deaths'] = np.nan
temp.loc[mask, 'Recovered'] = np.nan
# renaming columns
temp.columns = ['Country/Region', 'Date', 'New cases', 'New deaths', 'New recovered']
# merging new values
full_grouped = pd.merge(full_grouped, temp, on=['Country/Region', 'Date'])
# filling na with 0
full_grouped = full_grouped.fillna(0)
# fixing data types
cols = ['New cases', 'New deaths', 'New recovered']
full_grouped[cols] = full_grouped[cols].astype('int')
# 
full_grouped['New cases'] = full_grouped['New cases'].apply(lambda x: 0 if x<0 else x)

In [17]:
## Save cleaned data to a local .csv file
full_grouped.to_csv('COVID-19-time-series-clean-complete.csv', index = False)
full_grouped

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered
0,2020-01-22,Afghanistan,0,0,0.0,0.0,0,0,0
1,2020-01-22,Albania,0,0,0.0,0.0,0,0,0
2,2020-01-22,Algeria,0,0,0.0,0.0,0,0,0
3,2020-01-22,Andorra,0,0,0.0,0.0,0,0,0
4,2020-01-22,Angola,0,0,0.0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...
22873,2020-05-23,West Bank and Gaza,423,3,348.0,72.0,0,1,2
22874,2020-05-23,Western Sahara,6,0,6.0,0.0,0,0,0
22875,2020-05-23,Yemen,212,39,0.0,173.0,3,6,0
22876,2020-05-23,Zambia,920,7,336.0,577.0,0,0,0


### Make Interactive Line Plot with DateSlider

In [18]:
df_country = full_grouped[full_grouped['Country/Region'] == 'US']
df_country

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered
172,2020-01-22,US,1,0,0.0,1.0,0,0,0
358,2020-01-23,US,1,0,0.0,1.0,0,0,0
544,2020-01-24,US,2,0,0.0,2.0,1,0,0
730,2020-01-25,US,2,0,0.0,2.0,0,0,0
916,2020-01-26,US,5,0,0.0,5.0,3,0,0
...,...,...,...,...,...,...,...,...,...
22120,2020-05-19,US,1528568,91921,289392.0,1147255.0,20260,1574,6214
22306,2020-05-20,US,1551853,93439,294312.0,1164102.0,23285,1518,4920
22492,2020-05-21,US,1577147,94702,298418.0,1184027.0,25294,1263,4106
22678,2020-05-22,US,1600937,95979,350135.0,1154823.0,23790,1277,51717


In [19]:
import time
from datetime import datetime

import pandas as pd
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Slider, DateSlider,  HoverTool, Range1d
from bokeh.plotting import figure
from bokeh.themes import Theme
from bokeh.io import show, output_notebook

output_notebook()

Loading BokehJS ...

In [20]:
## Group by date and aggregate
global_count_df = full_grouped.groupby(['Date'])[['Confirmed', 'Deaths', 'Recovered']]
global_count_df = global_count_df.sum().reset_index()
global_count_df

,Date,Confirmed,Deaths,Recovered
0,2020-01-22,555,17,28.0
1,2020-01-23,654,18,30.0
2,2020-01-24,941,26,35.0
3,2020-01-25,1434,42,38.0
4,2020-01-26,2118,56,51.0
...,...,...,...,...
118,2020-05-19,4896757,323269,1787190.0
119,2020-05-20,4995737,328099,1844969.0
120,2020-05-21,5101689,332908,1894995.0
121,2020-05-22,5210082,338144,2001275.0


In [21]:
# global_count_df['Date'] =  global_count_df['Date'].dt.strftime('%d-%m-%Y')
global_count_df['Date'] = pd.to_datetime(global_count_df['Date'],dayfirst = True)
# global_count_df['Date'] = global_count_df['Date'].astype('datetime64[D]')
first_date = global_count_df['Date'][0]
last_date = global_count_df['Date'][len(global_count_df)-1]
global_count_df.set_index('Date', drop = True, inplace = True)
print('Global COVID Spread FROM:',first_date.strftime('%d-%m-%Y'),'TO:',last_date.strftime('%d-%m-%Y'))
global_count_df

Global COVID Spread FROM: 22-01-2020 TO: 23-05-2020


,Confirmed,Deaths,Recovered
Date,,,
2020-01-22,555,17,28.0
2020-01-23,654,18,30.0
2020-01-24,941,26,35.0
2020-01-25,1434,42,38.0
2020-01-26,2118,56,51.0
...,...,...,...
2020-05-19,4896757,323269,1787190.0
2020-05-20,4995737,328099,1844969.0
2020-05-21,5101689,332908,1894995.0


In [22]:
# Restrict data by date
global_count_df.loc[:'31-01-2020']

,Confirmed,Deaths,Recovered
Date,,,
2020-01-22,555,17,28.0
2020-01-23,654,18,30.0
2020-01-24,941,26,35.0
2020-01-25,1434,42,38.0
2020-01-26,2118,56,51.0
2020-01-27,2927,82,58.0
2020-01-28,5578,131,103.0
2020-01-29,6166,133,120.0
2020-01-30,8234,171,136.0


In [23]:
def bokeh_app(doc):
    df = global_count_df.copy()
    source = ColumnDataSource(data=df)
#     HoverTool(tooltips=[("Date", "@ENTRYDATE{%F}"), ("Transactions", "@Transactions")], formatters={"@ENTRYDATE":"datetime"}, mode='vline')
    hover = HoverTool(names=['covidcount'],tooltips=[("Date", "$x{%d-%m-%Y}"),("Count", "$y{0,000,000}")],formatters={"$x": "datetime"}, mode='mouse')
    hover.point_policy= "snap_to_data"
    plot = figure(title="@Dilip Rajkumar - Global COVID-19 Historic Spread - Bokeh Visualization",
                 plot_width=1200, plot_height=540, x_axis_type='datetime', y_axis_label='Nr. of Cases')
#     plot.x_range=DataRange1d(bounds=(None, 120))
    plot.x_range=Range1d(first_date, last_date)
    plot.y_range=Range1d(10, global_count_df['Confirmed'].max()*1.10)
    plot.add_tools(hover)
    plot.line('Date', 'Confirmed', source=source, color = 'orange',line_width=2, name = 'covidcount', legend_label="Nr. of ConfirmedCases", )
    plot.line('Date', 'Deaths', source=source, color = 'red',line_width=2,name = 'covidcount', legend_label="Nr. of Deaths")
    plot.line('Date', 'Recovered', source=source, color = 'green',line_width=2,name = 'covidcount', legend_label="Nr. of Recovered Cases")
    plot.legend.location = 'top_left'
    
    def callback(attr, old, new):
        date_value =  slider.value
        last_date = datetime.fromtimestamp(date_value/1000).strftime('%d-%m-%Y')
        df_trunc = df.loc[: last_date]
        source.data = df_trunc  

    slider = DateSlider(start=first_date, end=last_date, value='31-01-2020', step=1, title="Date")
    slider.on_change('value', callback)

    doc.add_root(column(slider, plot))

In [24]:
show(bokeh_app)

#### References:
1. [COVID 19 - Data Wrangling and Cleaning](https://towardsdatascience.com/covid-19-data-processing-58aaa3663f6)
2. [Bokeh AAPL Stocks - DateTime formatting](http://docs.bokeh.org/en/1.0.2/docs/user_guide/examples/tools_hover_tooltip_formatting.html)